In [1]:
import pandas as pd
import json
import os.path
import matplotlib.pyplot as plt
import collections
import matplotlib as m
import copy
import seaborn as sns
import re
import numpy as np
import os
from datetime import datetime

In [58]:
def maybe_parse_time(str):
    try:
        return datetime.strptime(str, '%H:%M:%S.%f')
    except ValueError:
        return False

In [106]:
maybe_parse_time('16:12:12.228744')

datetime.datetime(1900, 1, 1, 16, 12, 12, 228744)

In [109]:
def parse_call_time(line):
    matches = re.findall('([^ ]+) time: ([0-9]+) ?nanoseconds\s*',line)
    if not matches:
        return False
    elif len(matches) != 1 or len(matches[0]) != 2:
        raise Exception('Uh oh! ' + line, matches)
    
    (fname, ns) = matches[0]
    
    return (fname,int(ns))

In [143]:
def combine(raw_dir, csv_out):
    
    csvs = open(csv_out,'w')
    csvs.write('batch,version,case,local_seq,fname,ns\n')
    
    issues = open('issues.txt','w')
    any_issues = False
    
    for batch in os.listdir(raw_dir):
        for version in os.listdir(os.path.join(raw_dir,batch)):
            for case in os.listdir(os.path.join(raw_dir,batch,version)):
                filename = os.path.join(raw_dir,batch,version,case)
                file = open(filename,'r')
                
                lineno = 0
                local_seq = 0
                for line in file:
                    res = parse_call_time(line)
                    if not res:
                        if not line.startswith('Initializing') and not line.startswith('testing consistency') and not maybe_parse_time(line.strip()):
                            any_issues=True
                            issuestr = f'in {filename} SKIP [{lineno}]: {line[:-1]}'
                            print(issuestr)
                            issues.write(issuestr+'\n')
                        lineno += 1
                        continue
                    lineno += 1
                    (fname,ns) = res
                    
                    csvs.write(f'{global_seq},{batch},{version},{case},{local_seq},{fname},{ns}\n')
                    
                    local_seq+=1
    if any_issues:
        print('Some parsing issues!')
    else:
        print('No parsing issues :)')
        

In [144]:
combine('../output','results0.csv')

No parsing issues :)


In [145]:
df = pd.read_csv('results0.csv')

In [146]:
df

,batch,version,case,local_seq,fname,ns
35717,read-perf,basic,256_KB,0,getattr,1932816
35717,read-perf,basic,256_KB,1,getattr,2536427
35717,read-perf,basic,256_KB,2,getattr,1578917
35717,read-perf,basic,256_KB,3,getattr,1353814
35717,read-perf,basic,256_KB,4,open,18828004
...,...,...,...,...,...,...
35717,write-perf,perf,64_B,898,getattr,1426214
35717,write-perf,perf,64_B,899,read,4900
35717,write-perf,perf,64_B,900,write,5434755
35717,write-perf,perf,64_B,901,release,2913229


In [147]:
faceted = df.drop('local_seq',axis=1).groupby(['batch','version','case','fname'])

In [167]:
faceted.mean().to_csv('mean.csv')
faceted.min().to_csv('min.csv')
faceted.max().to_csv('max.csv')
faceted.median().to_csv('median.csv')
faceted.std().replace(np.nan,0).to_csv('sd.csv') # some tests don't have a mknod

In [150]:
df[df['fname']=='mknod']

,batch,version,case,local_seq,fname,ns
35717,write-perf,basic,256_KB,4,mknod,1220514
35717,write-perf,basic,4_KB,4,mknod,1980323
35717,write-perf,basic,64_B,4,mknod,2129321
35717,write-perf,perf,16_MB,5,mknod,1523017


In [160]:
mu = faceted.std().reset_index()
mu

,batch,version,case,fname,ns
0,read-perf,basic,256_KB,getattr,4.246443e+05
1,read-perf,basic,256_KB,open,1.765422e+06
2,read-perf,basic,256_KB,read,2.965404e+04
3,read-perf,basic,256_KB,release,7.422311e+03
4,read-perf,basic,4_KB,getattr,8.501409e+05
...,...,...,...,...,...
71,write-perf,perf,64_B,getattr,5.525077e+05
72,write-perf,perf,64_B,open,6.761510e+05
73,write-perf,perf,64_B,read,9.800824e+03
74,write-perf,perf,64_B,release,6.572584e+05


In [165]:
np.std([6])

0.0

In [142]:
df[['fname']].to_numpy()

array([[' getattr'],
       [' getattr'],
       [' getattr'],
       ...,
       [' write'],
       [' release'],
       [' release']], dtype=object)

In [126]:
df.loc[df['fname']=='getattr']

,batch,version,case,local_seq,fname,ns


In [133]:
df['fname'].to_numpy()

array([' getattr', ' getattr', ' getattr', ..., ' write', ' release',
       ' release'], dtype=object)